In [ ]:
import pandas as pd
pd.set_option("display.max_columns", 100)
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from haversine import haversine
CSV_DATA = Path().cwd().parent.parent / "data/0_extracted/processed/"

In [ ]:
def read_ride_csv(file_path:str, time_columns:list[str]=None):
    if time_columns is None: time_columns=['time']
    # Read in the CSV file for the Ride
    df = pd.read_csv(file_path)
    
    # guarantee the timestamps are datetime objects
    for time_col in time_columns:
        df[time_col] = pd.to_datetime(df[time_col])

    drop_columns = ['ride_id', 'track_id']
    df = df.drop(columns=drop_columns)

    add_null_columns = ['atemp', 'hr'] # if these columns don't already exist add them as np.null

    for col in add_null_columns:
        if col not in df.columns:
            df[col] = np.nan

    return df

In [ ]:
jan_ride_path = CSV_DATA / "January_no_sensor.csv"
apr_ride_path = CSV_DATA / "April_one_sensor.csv"
may_ride_path = CSV_DATA / "May_two_sensor.csv"
df_jan = read_ride_csv(jan_ride_path)
df_apr = read_ride_csv(apr_ride_path)
df_may = read_ride_csv(may_ride_path)

In [ ]:
def plot_state_column(df:pd.DataFrame, column:str, color_by:str='segment_id',figsize=(15,3),
                       ylims:list[float]=None, units:str='none') -> None:
    _ = plt.figure(figsize=figsize)
    _ = sn.lineplot(df, x='elapsed_time', y=column, hue=color_by, palette='hls', legend=False)
    _ = plt.grid()
    _ = plt.title(f'{column.upper()} Segments', fontsize=18)
    _ = plt.xlabel('Elapsed Time (seconds)', fontsize=14)
    _ = plt.ylabel(f'{column.upper()} ({units})', fontsize=14)
    if ylims is not None:
        _ = plt.ylim(ylims)

In [ ]:
class TimeUpsampler:
    def process(self, df:pd.DataFrame, time_gap_threshold:int=15, upsample:bool=False) -> pd.DataFrame:
        df = df.copy()
        df = self.enrich_time_data(df)
        df['elapsed_time'] = df['delta_time'].cumsum()
        df = self.label_continuous_segments(df, time_gap_threshold)
        if upsample:
            return self.upsample(df)
        else:
            return df

    def upsample(self, df:pd.DataFrame) -> pd.DataFrame:
        df_upsampled = self.normalize_sampling_rate(df)
        return df_upsampled

    @staticmethod
    def enrich_time_data(df:pd.DataFrame, time_column:str='time', fill_first:float=1.0):
        df = df.copy()
        # Temporarily get the number of seconds since Jan. 1, 1970 as the UTC timestamp
        df['time_utc'] = df[time_column].apply(lambda x: x.timestamp())
        
        # Calculate the row-wise difference in time (in seconds)
        df['delta_time'] = df['time_utc'].diff()
        
        # drop the temporary time column
        df.drop(['time_utc'], axis=1, inplace=True)
        
        # fill in the initial value of delta_time with @fill_first
        df['delta_time'] = df['delta_time'].fillna(fill_first)
        
        return df
    
    @staticmethod
    def label_continuous_segments(df: pd.DataFrame, time_gap_threshold:int=15):
        df = df.copy()
        # get the time gap indices
        # Calculate when the time discontinuities occur
        filt_time_jump = df['delta_time'] >= time_gap_threshold
        time_gap_indices = list(df.loc[filt_time_jump, 'time'].index)

        # intialize the initial segment_id. to be incremented for each region of continuous data
        segment_id_counter = 0
        # initialize the starting index of the first segment
        segment_start_index = 0

        for time_gap_index in time_gap_indices:
            # Assign the Segment ID
            df.loc[segment_start_index:time_gap_index-1, 'segment_id'] = segment_id_counter
            
            # update the segment_id counter and start index
            segment_id_counter += 1
            segment_start_index = time_gap_index
            
        # Since segment_id == -1 by default, this represents the final segment of activity once parsed
        df['segment_id'] = df['segment_id'].replace({-1:segment_id_counter})

        return df
    
    @staticmethod
    def upsample_and_interpolate(df: pd.DataFrame, time_column:str='time', method:str='linear', limit_direction:str='forward'):
        # Since the delta_time column is no longer needed to detect discontinuities,
        # Drop delta_time so we can rebuild it at a segment_id level
        df.drop(['delta_time'], axis=1, inplace=True)
        
        # set the timestamp as the index for the dataframe
        kwargs = dict(method=method, limit_direction=limit_direction)
        if method=='spline':
            kwargs['order']=2
        df = df.set_index(time_column).copy()
        df = df.resample('s').interpolate(**kwargs).reset_index()
        return df
    
    def normalize_sampling_rate(self, df: pd.DataFrame, partition_column:str='segment_id') -> pd.DataFrame:
        df = df.copy()
        functions_to_apply = [self.upsample_and_interpolate, self.enrich_time_data]
        for func in functions_to_apply:
            df = pd.concat(list(map(func, [df_group for _,df_group in df.groupby(partition_column)])), ignore_index=True).sort_index()
        return df

In [ ]:
upsampler = TimeUpsampler()
df_jan = upsampler.process(df_jan, upsample=True)
df_apr = upsampler.process(df_apr, upsample=True)
df_may = upsampler.process(df_may, upsample=True)

In [ ]:
from functools import reduce

class BasicEnricher:
    def process(self, df:pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        process_steps = [self.compute_distance, self.compute_heading, self.compute_speed, self.flag_cruising_rows, 
                         self.convert_elevation, self.compute_grade, self.compute_cumulative_elevation_changes]
        
        df_enriched = reduce(lambda df,func:func(df), process_steps, df)
        return df_enriched

    ##############################################################################################
    # Distance and Heading from Lat/Long
    ##############################################################################################
    @staticmethod
    def compute_distance(df, latitude='latitude', longitude='longitude', fill_first=0.0):
        df = df.copy()
        # Copy the previous values of Lat/Long to the current row for vectorized computation
        df['lat_old'] = df[latitude].shift()
        df['long_old'] = df[longitude].shift()
        
        # Grab the relevant columns for distance calculation
        df_gps = df[['lat_old', 'long_old', latitude, longitude]]
        
        # Define an anonymous function to execute over each row to calculate the distance between rows
        # Units should be in feet to prevent floating point precision issues for other calculations. (lower noise)
        haversine_distance = lambda x: max(0,haversine((x.iloc[0], x.iloc[1]), (x.iloc[2], x.iloc[3]), unit='ft'))
        
        # Create the distance column, making sure to apply the function row-by-row
        df['delta_dist_ft'] = df_gps.apply(haversine_distance, axis=1)
        df['delta_dist_ft'] = df['delta_dist_ft'].fillna(fill_first)
        
        # Remove the old latitude and longitude columns
        df.drop(['lat_old','long_old'], axis=1, inplace=True)
        return df
    
    @staticmethod
    def compute_heading(df, latitude='latitude', longitude='longitude'):
        df = df.copy()
        # Copy the previous values of Lat/Long to the current row for vectorized computation
        df['lat_old'] = df[latitude].shift()
        df['long_old'] = df[longitude].shift()
        
        # Grab the relevant columns for distance calculation
        df_gps = df[['lat_old', 'long_old', latitude, longitude]]
        
        # Define an anonymous function to execute over each row to calculate the angle with North as 0 degrees
        # NOTE: we use "delta_lat / delta_long" to ensure that North = 0 degrees
        rad2deg = 180.0 / np.pi
        heading = lambda x: rad2deg * np.arctan2((x.iloc[2]-x.iloc[0]), (x.iloc[3]-x.iloc[1])) # atan(delta_lat / delta_long)
        
        # Create the distance column, making sure to apply the function row-by-row
        df['heading'] = df_gps.apply(heading, axis=1)
        df['heading'] = df['heading'].apply(lambda x: x + 360.0*(1-np.sign(x))/2) # correct for negative angles
        
        # Remove the old latitude and longitude columns
        df.drop(['lat_old','long_old'], axis=1, inplace=True)
        return df

    ##############################################################################################
    # Speed Enrichments
    ##############################################################################################
    @staticmethod
    def compute_speed(df):
        df = df.copy()
        feet_to_miles = 1.0 / 5280.0
        miles_per_second_2_MPH = 3600.0 / 1.0 # conversion factor
        df['speed'] = miles_per_second_2_MPH * (feet_to_miles*df['delta_dist_ft']) / df['delta_time']
        return df

    @staticmethod
    def flag_cruising_rows(df, start_threshold_mph:float=8.0, stop_threshold_mph:float=5.0):
        """
        Scmitt Trigger to implement a hysteresis state machine for determining a state
        """
        df = df.copy()
        df['is_cruising'] = False

        for k in range(1, df.shape[0]):
            previous_state = df.loc[k-1,'is_cruising']
            current_speed = df.loc[k,'speed']
            if (previous_state==False) & (current_speed >= start_threshold_mph):
                df.loc[k,'is_cruising'] = True # rising threshold surpassed
            elif (previous_state==True) & (current_speed < stop_threshold_mph):
                df.loc[k,'is_cruising'] = False # rising threshold surpassed
            else:
                # if there is no change, propogate the previous state
                df.loc[k,'is_cruising'] = df.loc[k-1,'is_cruising']
        return df

    ##############################################################################################
    # Elevation Enrichments
    ##############################################################################################
    @staticmethod
    def convert_elevation(df):
        df = df.copy()
        meters_to_feet = 3.281
        df['elevation'] = df['elevation'] * meters_to_feet
        return df

    @staticmethod
    def compute_grade(df):
        df = df.copy()
        fill_first = 0.0
        df['delta_ele_ft'] = df['elevation'].diff()
        df['delta_ele_ft'] = df['delta_ele_ft'].fillna(fill_first)
        df['grade'] = 100.0 * (df['delta_ele_ft'] / df['delta_dist_ft'])
        df.loc[0,'grade'] = 0.0 # initialize and assumed 0% slope as the starting point--representative of a typical parking lot
        df.loc[~np.isfinite(df['grade']),'grade'] = np.nan

        # fill in nulls where delta_dist==0.0 by interpolating the value. If you stop on a hill, your grade should carry forward
        df['grade'] = df['grade'].interpolate('linear')

        # Constrain Grade to be within the typical +/- 15 % all riders deal with. We'll use 18% as thresholds
        df['grade_saturated'] = df['grade'].apply(lambda g: min(g,18)).apply(lambda g: max(g,-18))

        return df

    @staticmethod
    def compute_cumulative_elevation_changes(df, fill_first=0.0):
        df = df.copy()
        
        # create an elevation difference
        df['delta_ele'] = df['elevation'].diff()
        df['delta_ele'] = df['delta_ele'].fillna(fill_first)
        
        # create delta ascent and delta descent columns
        df['delta_ascent'] = df.loc[df['delta_ele']>=0, 'delta_ele']
        df['delta_descent'] = df.loc[df['delta_ele']<=0, 'delta_ele']
        
        # create the cumulative versions
        df['elapsed_ascent'] = df['delta_ascent'].cumsum()
        df['elapsed_ascent'] = df['elapsed_ascent'].interpolate() # fill in any blanks
        df['elapsed_descent'] = df['delta_descent'].cumsum()
        df['elapsed_descent'] = np.abs(df['elapsed_descent'].interpolate()) # fill in any blanks
        
        # create the total elevation change column
        df['elapsed_elevation'] = df['elapsed_ascent'] + df['elapsed_descent']
            
        # drop the elevation differences
        df.drop(['delta_ele','delta_ascent','delta_descent'], axis=1, inplace=True)
        
        return df

class PowerEstimator:
    def __init__(self, power_params:dict, speed_col:str='filt_speed', grade_col:str='filt_grade_saturated')->None:
        self.power_params = power_params
        self.speed_col = speed_col
        self.grade_col = grade_col

    def estimate_power(self, df:pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        return self.get_instantaneous_power(df, self.power_params, self.speed_col, self.grade_col)

    @staticmethod
    def get_instantaneous_power(df:pd.DataFrame, power_params:dict, speed_column:str='speed', grade_column:str='grade_saturated'):
        cols_to_drop_later = ['grade_radians','speed_MpS','F_grav','F_fric','F_drag', 'F_sum', 
                            'total_speed']
        params = power_params

        # Convert the terrain slope into radians
        df['grade_radians'] = np.arctan(df[grade_column]/100)
        
        # Convert the speed units into meters per second
        mph2MpS = 0.44704 # 1 MPH = 0.44704 m/s
        df['speed_MpS'] = mph2MpS * df[speed_column]
        
        # Get the total speed component with wind (placeholder)
        df['total_speed'] = df['speed_MpS']
        
        # Calculate the individual forces
        pounds_to_kilograms = 0.453592
        total_mass = pounds_to_kilograms * sum(list(params['weight'].values()))
        df['F_grav'] = total_mass*params['gravity'] * np.sin(df['grade_radians'])
        df['F_fric'] = params['mu_rr']*total_mass*params['gravity'] * np.cos(df['grade_radians'])
        full_coefficient = 0.5 * params['rho_air'] * params['area'] * params['c_drag']
        df['F_drag'] = (full_coefficient) * np.power(df['total_speed'], 2) # k(v)^2
        
        # Sum the forces
        df['F_sum'] = df['F_drag'] + df['F_grav'] + df['F_fric']
        
        # Calculate the non-negative power delivered by the ride (set Power=0 for F_sum <0)
        df['inst_power'] = (1.0/params['eta_dt']) * df['F_sum'] * df['speed_MpS'] 
        df.loc[df['inst_power']<0,'inst_power'] = 0 # coasting when sum of forces is negative (no input power)

        df.drop(columns=cols_to_drop_later, inplace=True)
        
        return df

In [ ]:
enricher = BasicEnricher()
df_jan = enricher.process(df_jan)
df_apr = enricher.process(df_apr)
df_may = enricher.process(df_may)

In [ ]:
from scipy import signal
import numpy as np

class NoiseFilter:
    def __init__(self, filter_type:str='hann', filter_order:int=21):
        if filter_type=='hann':
            fir_filter = signal.windows.hann(filter_order)
        elif filter_type=='blackman-harris':
            fir_filter = signal.windows.blackmanharris(filter_order)
        elif filter_type=='rect':
            fir_filter = signal.windows.boxcar(filter_order)
        self.fir_filter = fir_filter

    def filter_columns(self, df:pd.DataFrame, columns:list[str]) -> pd.DataFrame:
        df = df.copy()
        for col in columns:
            df = self.apply_filter(df, col, self.fir_filter)
        return df 

    @staticmethod
    def apply_filter(df:pd.DataFrame, signal_column:str, fir_filter:np.ndarray) -> pd.DataFrame:
        signal_list = list(df[signal_column].values)
        filtered_signal = signal.convolve(signal_list, fir_filter, mode='same') / sum(fir_filter)
        
        df['filt_'+signal_column] = np.nan
        if len(signal_list) == len(filtered_signal):
            df['filt_'+signal_column] = filtered_signal
        else:
            df.loc[1:,'filt_'+signal_column] = filtered_signal
            df.loc[0,'filt_'+signal_column] = df.loc[1,'filt_'+signal_column] # backfill
        return df



In [ ]:
noise_filter = NoiseFilter()
columns_to_filter = ['hr', 'speed', 'grade', 'grade_saturated']
df_jan = noise_filter.filter_columns(df_jan, columns_to_filter)
df_apr = noise_filter.filter_columns(df_apr, columns_to_filter)
df_may = noise_filter.filter_columns(df_may, columns_to_filter)


In [ ]:
power_params = {'weight': {'rider':220, # needs to be converted lb-->kg
                   'bike':25,
                   'bags':5
                  }, 
                  'area': 0.4635862, # m^2
                  'mu_rr': 0.005, # coefficient of rolling friction
                  'c_drag': 0.95, # coefficient of drag
                  'rho_air': 1.2, # kg/m^3 air density
                  'eta_dt': 0.96, # efficiency of drive train
                  'gravity': 9.8 # m/s^2
                 }

pwr_estimator = PowerEstimator(power_params, speed_col='filt_speed', grade_col='filt_grade_saturated')

In [ ]:
data = [{'ride_id':555, 'b_weight':50, 'a_weight':100, 'c_weight':10, 'other':20000},
{'ride_id':222, 'b_weight':4, 'a_weight':5, 'c_weight':6, 'other':777}]
df = pd.DataFrame(data)


In [ ]:
df_jan = pwr_estimator.estimate_power(df_jan)
df_apr = pwr_estimator.estimate_power(df_apr)
df_may = pwr_estimator.estimate_power(df_may)

In [ ]:
plot_state_column(df_apr,'inst_power', units='W')

In [ ]:
SECRETS_CONFIG = Path().cwd().parent.parent / "etl/sensitive_config/secrets.yaml"
import yaml

with open(SECRETS_CONFIG, 'r') as f:
    config = yaml.safe_load(f)

In [ ]:
class PrivacyZoner():
    def __init__(self, privacy_zone_config:dict):
        self.df = None
        self.privacy_zone_config = privacy_zone_config
        self.df_privacy = None
        self.temporary_prox_columns = []

    def process(self, df:pd.DataFrame) -> pd.DataFrame:
        self.df = df.copy()
        self._read_privacy_zones()
        self._calculate_proximities()
        self._remove_violation_gps_data()
        self._drop_temporary_prox_columns()
        return self.df

    ################################################################
    # PROCESS METHODS
    ################################################################

    def _read_privacy_zones(self):
        data = []
        for private_location in self.privacy_zone_config.keys():
            datum = {'name':private_location, 'latitude':self.privacy_zone_config[private_location]['lat'],
                     'longitude':self.privacy_zone_config[private_location]['long'], 
                     'privacy_radius':self.privacy_zone_config[private_location]['scrub_radius']}
            data.append(datum)
        self.df_privacy = pd.DataFrame(data)

    def _calculate_proximities(self):
        for private_location in range(self.df_privacy.shape[0]):
            # get the relevant parameters
            dist_name = 'prox_' + self.df_privacy.loc[private_location, 'name']
            latitude = self.df_privacy.loc[private_location, 'latitude']
            longitude = self.df_privacy.loc[private_location, 'longitude']
            
            # calculate the proximity
            self.df[dist_name] = self._get_proximity_to_address(latitude, longitude)
            self.temporary_prox_columns.append(dist_name)

    def _remove_violation_gps_data(self):
        for private_location in range(self.df_privacy.shape[0]):
             # get the relevant parameters
            dist_name = 'prox_' + self.df_privacy.loc[private_location, 'name']
            privacy_radius = self.df_privacy.loc[private_location, 'privacy_radius']

            filt_violation = self.df.loc[:,dist_name] <= privacy_radius
            self.df.loc[filt_violation, 'latitude'] = np.nan
            self.df.loc[filt_violation, 'longitude'] = np.nan

    def _drop_temporary_prox_columns(self):
        self.df.drop(self.temporary_prox_columns, axis=1, inplace=True)

    ################################################################
    # HELPER METHODS
    ################################################################

    def _get_proximity_to_address(self, addr_latitude, addr_longitude):
        df_gps = self.df[['latitude', 'longitude']]
        
        # Define an anonymous function to execute over each row to calculate the distance between rows
        haversine_distance = lambda x: haversine((x.iloc[0], x.iloc[1]), (addr_latitude, addr_longitude), unit='mi')
        
        # Create the distance column, making sure to apply the function row-by-row
        proximity = df_gps.apply(haversine_distance, axis=1)
        
        return proximity

In [ ]:
SECRETS_CONFIG = Path().cwd().parent.parent / "etl/sensitive_config/secrets.yaml"
import yaml

with open(SECRETS_CONFIG, 'r') as f:
    config = yaml.safe_load(f)

zoner = PrivacyZoner(config)

In [ ]:
df_jan = zoner.process(df_jan)
df_apr = zoner.process(df_apr)
df_may = zoner.process(df_may)

In [ ]:
df_may.head()

In [ ]:
df_apr.head()

In [30]:
hex(hash(df_apr.time[0].timestamp()))

'0x67faa109'